In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
diamonds_train = pd.read_csv('outputs/df_train.csv')
diamonds_train_price = pd.read_csv('outputs/df_train_price.csv')
diamonds_predict=pd.read_csv('outputs/df_test.csv')
submit=pd.read_csv('inputs/sample_submission.csv')

In [3]:
diamonds_train.head()

,carat,cut_numeric,color_numeric,clarity_numeric,depth,table,x,y,z
0,-1.051355,-0.810564,0.352161,-0.033632,0.736126,-0.650176,-1.283716,-1.236438,-1.180531
1,-0.966764,0.982588,0.940769,-0.641579,0.596363,-1.097476,-1.132102,-1.080061,-1.039596
2,-0.839879,-0.810564,1.529377,-0.641579,-1.010919,2.033618,-0.918059,-0.854183,-0.969129
3,-0.839879,0.086012,-0.825055,0.574315,0.037308,0.781181,-0.900222,-0.862870,-0.870475
4,0.217504,-0.810564,1.529377,-0.641579,-0.521746,2.480917,0.330526,0.344710,0.271096


In [4]:
diamonds_predict.head()

,carat,cut_numeric,color_numeric,clarity_numeric,depth,table,x,y,z
0,3.267015,0.978152,-1.416151,-1.232414,-0.666318,-1.554540,2.620734,2.539800,2.446736
1,2.596725,0.978152,-0.830504,-1.232414,0.168508,-0.658049,2.175567,2.234832,2.230593
2,-0.608098,0.978152,-1.416151,-0.627731,-0.040199,-1.554540,-0.495431,-0.491942,-0.492811
3,-1.047976,0.978152,-1.416151,-0.627731,-0.318474,-0.658049,-1.261117,-1.263332,-1.285335
4,0.334497,-2.592365,-0.830504,-0.023048,4.899190,-0.658049,0.332578,0.216660,0.890505


# Split train and test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diamonds_train, diamonds_train_price, test_size=0.03)

In [ ]:
X_train.head()

In [ ]:
y_test.shape

# GradientBoostingRegressor

In [5]:
from sklearn.ensemble import GradientBoostingRegressor

In [6]:
reg = GradientBoostingRegressor(n_estimators = 100, loss='huber', min_samples_leaf=6, max_depth=7,
                                max_leaf_nodes=250, min_samples_split=45
                                )

In [7]:
reg.fit(diamonds_train, diamonds_train_price)

/home/must4in3/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='huber',
                          max_depth=7, max_features=None, max_leaf_nodes=250,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=6, min_samples_split=45,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [8]:
y_pred_GBR= reg.predict(diamonds_predict)

In [ ]:
y_test_GBR = y_test.copy()
y_test_GBR['y']= y_pred_GBR

In [ ]:
y_test_GBR['Price-y'] = abs(y_test_GBR['price']- y_test_GBR['y'])
y_test_GBR.describe()

In [9]:
df_y = pd.DataFrame(y_pred_GBR)

In [10]:
df_y['id'] = df_y.index

In [11]:
df_y['price'] = df_y[0]

In [12]:
df_y = df_y[['id', 'price']]

In [13]:
df_y

,id,price
0,0,15147.743856
1,1,16482.270983
2,2,980.650204
3,3,475.708763
4,4,3137.903308
...,...,...
13480,13480,2546.695085
13481,13481,5104.659633
13482,13482,2436.329104
13483,13483,1022.014927


In [14]:
df_y.to_csv('gaussian_br100-huber-6-7-250-45(4).csv', index=False)

In [ ]:
sns.set()
fig, ax = plt.subplots()
ax.scatter( y_test_GBR['price'], y_test_GBR['y'], alpha=.3)
ax.plot([y_test_GBR.min(), y_test_GBR.max()], [y_test_GBR.min(), y_test_GBR.max()], 'k--', lw=4)
ax.set_xlabel('measured')
ax.set_ylabel('predicted')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
print(f'RMSE={mean_squared_error(y_pred_GBR, y_test)**0.5}')